In [11]:
import pandas as pd
import numpy as np
import html
import re
from bs4 import BeautifulSoup
import html2text
import os

In [2]:
def html_to_text(html_content):
    # Create an HTML2Text object
    h = html2text.HTML2Text()
    
    # Configure the converter
    h.ignore_links = True
    h.ignore_images = True
    h.ignore_emphasis = True
    h.body_width = 0  # No line wrapping
    
    # Convert HTML to text
    text = h.handle(html_content)
    
    # Clean up the text
    text = re.sub(r'\n\s*\n', '\n\n', text)  # Remove extra blank lines
    text = re.sub(r' +', ' ', text)  # Remove extra spaces
    text = text.strip()  # Remove leading/trailing whitespace
    
    return text

In [5]:
def extract_mda(text):
    
    text_lower = text.lower()


    start_pattern = r"item\s*7\.\s*.*?discussion\s*and\s*analysis"
    #start_pattern = r"item\s*7\.?.*?discussion\s*and\s*analysis"
    end_pattern = r"item[^\n]*\s*7a\.?[\s\n]*quantitative and qualitative"
    
    matching_pattern = r'item 6'
    
 
    match = re.search(f"{start_pattern}(.*?){end_pattern}", text_lower, re.DOTALL)
    
    
    if match:
        
        start = match.start()
        end = match.end()
        
       
        original_content = text[start:end]
        
        matches = re.search(matching_pattern, original_content, re.S | re.I)
        
        if matches:
            
            start_item6 = matches.start()
            original_content = original_content[start_item6:]
        
        return original_content.strip()
    else:
        return "not find"




In [4]:
def clean_text_basic(text):
    
    text = re.sub(r'^\d+\s*$', '', text, flags=re.M)

    text = re.sub(r'^\s*\* \* \*\s*$', '', text, flags=re.M)
    
    text = re.sub(r'^\s*Table of Contents\s*$', '', text, flags=re.I | re.M)
    
    
    return text

#### sample code -test one by one

In [10]:
# Read the file content
with open(your_test_filing_path, 'r', encoding='utf-8') as file:
    file_content = file.read()

# Parse the response (the XML flag works better than HTML for 10Ks)
soup = BeautifulSoup(file_content, 'lxml')

for table in soup.find_all('table'):
    if not table.find('span', string=re.compile('●')):
        table.decompose()
# Convert soup object to string
soup_str = str(soup)

# Remove all superscripts like (1)
soup_str = re.sub(r'\(\d+\)', '', soup_str)

# Remove extra spaces before punctuation
soup_str = re.sub(r'\s+([,.])', r'\1', soup_str)

# Convert the string back to BeautifulSoup object for further processing
soup = BeautifulSoup(soup_str, 'lxml')

for filing_document in soup.find_all('document'): # The document tags contain the various components of the total 10K filing pack
    
    # The 'type' tag contains the document type
    #document_type = filing_document.type.find(text=True, recursive=False).strip()
    document_type = filing_document.type.find(string=True, recursive=False).strip()

    if document_type == "10-K": # Once the 10K text body is found
        
        TenKtext = html_to_text(str(filing_document))
        
        
        
        '''
        
        #-------------------matching regular expression--------------------------------
        TenKtext_lower = TenKtext.lower()
        
        start_pattern = r"item\s*7\.\s*.*?discussion\s*and\s*analysis"
        
        end_pattern = r"item[^\n]*\s*7a\.?[\s\n]*quantitative and qualitative"
        
        match = re.search(f"{start_pattern}(.*?){end_pattern}", TenKtext_lower, re.DOTALL)
        
        
        if match:
            start = match.start()
            end = match.end()
            
            original_content = TenKtext[start:end]
            
            #print(original_content)
            
            pattern = r'Item 6\. \[Reserved\](.*)'
            
            matches = re.search(pattern, original_content, re.S | re.I)
            
            if matches:
                extracted_text_twice = matches.group(1).strip() 
                print(extracted_text_twice)
            else:
                print("still not found")
        '''
           
        #print(TenKtext)
        mda_result = extract_mda(TenKtext)
        mda_result = clean_text_basic(mda_result)
        print(mda_result)
        

        

Item 7.Management’s Discussion and Analysis of Financial Condition and Results of Operations

The following discussion and analysis should be read in conjunction with our consolidated financial statements and the related notes thereto included in Item 8 and the Risk Factors included in Item 1A of Part I of this Annual Report on Form 10-K. All information presented herein is based on our fiscal calendar. Unless otherwise stated, references in this Annual Report on Form 10-K to particular years or quarters refer to our fiscal years ended in July and the associated quarters of those fiscal years. We assume no obligation to revise or update any forward-looking statements for any reason, except as required by law.

We have elected to omit discussion on the earliest of the three years covered by the consolidated financial statements presented. Refer to Item 7. Management’s Discussion and Analysis of Financial Condition and Results of Operations located in our Form 10-K for the fiscal year en

#### formal code

In [12]:

# Define input and output directories
input_dir = your_folder_saved_raw_txt_filings  #fill in your own folder
output_dir = your_folder_to_save_MDA              #fill in your own folder

# Loop through each file in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith(".txt"):
        input_filepath = os.path.join(input_dir, filename)
        output_filepath = os.path.join(output_dir, filename)

        # Read the file content
        with open(input_filepath, 'r', encoding='utf-8') as file:
            file_content = file.read()

        # Parse the response (the XML flag works better than HTML for 10Ks)
        soup = BeautifulSoup(file_content, 'lxml')

        for table in soup.find_all('table'):
            if not table.find('span', string=re.compile('●')):
                table.decompose()
        
        # Convert soup object to string
        soup_str = str(soup)

        # Remove all superscripts like (1)
        soup_str = re.sub(r'\(\d+\)', '', soup_str)

        # Remove extra spaces before punctuation
        soup_str = re.sub(r'\s+([,.])', r'\1', soup_str)

        # Convert the string back to BeautifulSoup object for further processing
        soup = BeautifulSoup(soup_str, 'lxml')

        for filing_document in soup.find_all('document'):
            #document_type = filing_document.type.find(text=True, recursive=False).strip()
            document_type = filing_document.type.find(string=True, recursive=False).strip()

            if document_type == "10-K":
                TenKtext = html_to_text(str(filing_document))
                mda_result = extract_mda(TenKtext)
                mda_result = clean_text_basic(mda_result)
                
                # Save the mda_result to the output file
                with open(output_filepath, 'w', encoding='utf-8') as output_file:
                    output_file.write(mda_result)


#### deal with the cases that failed

In [27]:
saved_folder_path = your_folder_to_save_MDA                      #fill in your own folder
saved_null_list = []
# Iterate through the files in the folder
for filename in os.listdir(saved_folder_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(saved_folder_path, filename)
        # Check if the file size is less than 5 KB (5 KB = 5120 bytes)
        if os.path.getsize(file_path) < 10000:
            save_null_filepath = your_folder_saved_raw_txt_filings + '\\' + filename  #fill in your own folder
           
            saved_null_list.append(save_null_filepath)


In [28]:
start_pattern = r"(?i)item\s*7\b"
end_pattern = r"(?i)item\s*7a\b"


for txt_files in saved_null_list:
    with open(txt_files, 'r', encoding='utf-8') as file:
        file_content = file.read()
        
    soup = BeautifulSoup(file_content, 'lxml')
    
    for table in soup.find_all('table'):
        if table.find('span', string=re.compile('●')) or re.search(r'item\s*7(?:a)?', table.get_text(), re.IGNORECASE):
            continue  
        else:
            table.decompose() 
            
    soup_str = str(soup)
    soup_str = re.sub(r'\(\d+\)', '', soup_str)
    soup_str = re.sub(r'\s+([,.])', r'\1', soup_str)
    soup = BeautifulSoup(soup_str, 'lxml')
    
    for filing_document in soup.find_all('document'):
        
        document_type = filing_document.type.find(string=True, recursive=False).strip()
        
        if document_type == "10-K": 
            TenKtext = html_to_text(str(filing_document))
            
            #mda_result = extract_mda(TenKtext)
            #mda_result = clean_text_basic(mda_result)
            
            #print(mda_result)
            #print(TenKtext)
            
            
            text = TenKtext
            
            start_matches = list(re.finditer(start_pattern, text))
            
            contents = []
            
            for start_match in start_matches:
                
                start_index = start_match.end()
                next_item_match = re.search(r"(?i)item\s*7a\b|item\s*7\b", text[start_index:])
                
                if next_item_match:
                    matched_text = next_item_match.group().lower()
                    
                    if '7a' in matched_text:
                        end_index = start_index + next_item_match.start()
                        content = text[start_index:end_index].strip()
                        contents.append(content)
                    else:
                        continue
                        
                else:
                    break
            
            saved_filepath = your_folder_to_save_MDA_in_step_two + '\\' + txt_files.split('\\')[-1]         #fill in your own folder
            if contents:
                longest_content = max(contents, key=len)
                longest_content = clean_text_basic(longest_content)
                
                with open(saved_filepath, 'w', encoding='utf-8') as file:
                    file.write(longest_content)
            else:
                with open(saved_filepath, 'w', encoding='utf-8') as file:
                    file.write("no findings on matched results")

